# 3) Proprietary Data
### Preparation step
First we need to import necessary libraries and configuration from the _config.ini_ file

In [ ]:
import datetime
import requests
import json
import configparser

#config
config = configparser.ConfigParser()
config.read('C:\\RDMSPython\\config.ini')
api = config['RDMS']['Api']
headers = { 'Authorization' : config['RDMS']['Key'] }

### 3.1) RegisterPropCurve
Uses the _v1/RegisterPropCurve_ RDMS REST API endpoint. The script takes parameters 
provided in JSON format in order to create a new prop curve.

In [ ]:
#register prop curve
newCurve = {
  'curveName': 'My Sample Curve',
  'commodity': 'Gasoline',
  'unitOfMeasure': 'MW',
  'valueFrequency': 'Months (1)',
  'variable': 'Gasoline Price',
  'status': 'Forecast',
  'timeZone': 'UTC',
  'securityGroup': 'PUBLIC'
}
result = requests.post(api + '/RegisterPropCurve', json=newCurve, headers=headers, verify=True)

if result.status_code == 200:
    body = result.json()
    print('Created ' + str(body['curveID']))
else:
    print('HTTP ' + str(result.status_code) + ' failed to save')
    exit()

### 3.2) PropCurveImport
Uses the _v1/PropCurveImport_ RDMS REST API endpoint. The script takes parameters 
provided in JSON format in order to populate prop curve values.

In [ ]:
resultPoints = []
minDate = "2020-04-01"  # input start date
maxDate = "2020-09-01"  # input end date
#populate a list of months
month_list = [i.strftime("%Y-%m-%d") for i in pd.date_range(start=minDate, end=maxDate, freq='MS')]
#build the curve points list
for i in range(len(month_list)):
    resultPoints.append({
            "valueDate": dateutil.parser.parse(month_list[i]).isoformat(),
            "value": (random.randrange(10,50))
        })

curveId = 910000024
newForecast = {
  "curveID": curveId,
  "scenarioID": 0,
  "forecastDate": datetime(2020, 3, 2, tzinfo=timezone.utc).isoformat(),
  "values": resultPoints
}

result = requests.post(api + '/PropCurveImport', json=newForecast, headers=headers, verify=True)

if result.status_code == 200:
    print('Curve data saved')
else:
    print('HTTP ' + str(result.status_code) + ' failed to save')